In [7]:
import numpy as np
import pandas as pd
import glob
import seaborn

In [8]:
def insitu_mergeCSV(path):
    all_filenames = glob.glob(path + "/*.csv")
    merge = pd.concat([pd.read_csv(f,usecols = ['StationCode','DateTimeStamp', 'Temp', 'DO_mgl']) for f in all_filenames ])
    return merge


def mergeCSV(path):
    all_filenames = glob.glob(path + "/*.csv")
    merge = pd.concat([pd.read_csv(f) for f in all_filenames ])#, dtype = {'siteID':str}
    
    return merge
  
def pull_clean (data, dswe = 1, minPix = 5, maxCloud = 50, minRef = 0, maxRef = 10000):
    data =data.rename(columns={"Blue": "blue", "Green": "green", "Red": "red", "Nir": "nir", "Swir1": "swir1", "Swir2": "swir2",
                               "Aerosol": "aerosol", "system:index": "LS_ID"})
    
    data = data.dropna(subset=['blue'])
    
    data = data[(data['blue'] <= maxRef) & (data['blue'] > minRef)&
          (data['green'] <= maxRef) & (data['green'] > minRef)&
          (data['red'] <= maxRef) & (data['red'] > minRef)&
          (data['nir'] <= maxRef) & (data['nir'] > minRef)&
          (data['swir1'] <= maxRef) & (data['swir1'] > minRef)&
          (data['swir2'] <= maxRef) & (data['swir2'] > minRef )]


    data[['sat', 'pathrow', 'date', 'junk']]  = data['LS_ID'].str.split('L', n = 1, expand = True)[1].str.split('_', n =4 , expand = True)
    data = data.drop(columns=['junk','LS_ID'])
    data['sat'] = 'L'+data['sat'] 
    data["date"] = pd.to_datetime(data["date"]).dt.date
    data['datetime']= pd.to_datetime(data['time'], unit='ms')
    
    return data

In [ ]:
path = r"D:\Estuary Temperature\insitu data\old 1\insitu data data\csv"
obs = insitu_mergeCSV(path)
obs['StationCode'] = obs['StationCode'].str.lower().str.strip()
good_sites = pd.read_csv(r"D:\Estuary Temperature\insitu data\matchup_v2.csv")
good_sites = good_sites['StationCode'].drop_duplicates()
obs=obs[obs.StationCode.isin(good_sites)]

obs['sampleddatetime'] = pd.to_datetime(obs['DateTimeStamp'])
obs['sampledhour'] = obs['sampleddatetime'].dt.hour 
obs = obs[(obs.sampledhour < 12) & (obs.sampledhour > 8)]
obs['year'] = obs['sampleddatetime'].dt.year
obs['month'] = obs['sampleddatetime'].dt.month
obs = obs[(obs.Temp > 0) & (obs.Temp < 35)]
obs['key'] = obs ['StationCode']+obs ['year'].astype(str)+obs['month'].astype(str)

threshold = obs.groupby(['StationCode', 'year', 'month'])['Temp'].describe()[['mean','max', 'min', '25%', '75%']].reset_index()
threshold['key'] = threshold ['StationCode']+threshold ['year'].astype(str)+threshold ['month'].astype(str)
threshold ['iqr'] = threshold ['75%'] -threshold ['25%']
threshold ['iqr_upper_bound'] = threshold['75%'] + 1.5*threshold ['iqr']
threshold ['iqr_lower_bound'] = threshold['25%'] - 1.5*threshold ['iqr']
threshold = threshold.rename(columns={"max":"station_monthly_max", "min":"station_monthly_min"})
threshold = threshold[['key', 'station_monthly_max', "station_monthly_min", 'iqr_upper_bound', 'iqr_lower_bound']]

obs = obs.merge(threshold, on='key', how='left')
obs = obs[(obs.Temp < obs.iqr_upper_bound) & (obs.Temp > obs.iqr_lower_bound)]
obs = obs[['StationCode','sampleddatetime','year','month','station_monthly_max','station_monthly_min','Temp']]

siteInfo = pd.read_csv(r"D:\Estuary Temperature\insitu data\old 1\timezone.csv")
siteInfo['Name'] = siteInfo['Name'].str.lower()
siteInfo = siteInfo[['Name', 'tzid']].drop_duplicates()
siteInfo = siteInfo.rename(columns={"Name": "StationCode"})

obs = obs.merge(siteInfo, how='left', on='StationCode')

obs['unitysampledatetime'] = obs.groupby('tzid')['sampleddatetime'].transform(lambda x: x.dt.tz_localize(x.name, ambiguous=True))
obs['unitysampledatetime'] = pd.to_datetime(obs['unitysampledatetime'] , utc=True)
obs['unitysampledatetime'] = pd.to_datetime(obs.unitysampledatetime).dt.tz_localize(None)

obs['rn_obs'] = obs.index
obs[['rn_obs', 'StationCode', 'unitysampledatetime']].to_csv("obs_rn.csv", index = False)
obs.to_csv("obs_filtered.csv", index = False)

In [ ]:
path = r"D:\Estuary Temperature\insitu data\old 1\sr\csv"
sr = mergeCSV(path)
sr = pull_clean (sr)
sr['StationCode'] = sr['Name'].str.lower()
sr['year'] = sr['datetime'].dt.year
sr['month'] = sr['datetime'].dt.month
sr['key'] = sr ['StationCode']+sr ['month'].astype(str)
sr['LS_Temp'] = sr['TIR1']-273.15
sr = sr[(sr.LS_Temp < 35) & (sr.LS_Temp > 0) & (sr.ST_CDIST > 300) & (sr.ST_QA < 273.15)]
sr = sr[['aerosol', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'TIR1', 'sd_NirSD', 'pixel_qa', 'clouds', 'ST_CDIST', 'ST_QA', 
         'dswe','pCount_dswe3', 'pCount_dswe1', 'time', 'siteID','sat', 'pathrow', 'date', 'datetime', 'StationCode', 
         'year', 'month','key', 'LS_Temp']]

threshold = sr.groupby(['StationCode','month'])['LS_Temp'].describe()[['mean','max', 'min', '25%', '75%']].reset_index()
threshold['key'] = threshold ['StationCode']+threshold ['month'].astype(str)
threshold ['iqr'] = threshold ['75%'] -threshold ['25%']
threshold ['iqr_upper_bound'] = threshold['75%'] + 1.5*threshold ['iqr']
threshold ['iqr_lower_bound'] = threshold['25%'] - 1.5*threshold ['iqr']
threshold = threshold.rename(columns={"max":"station_monthly_max", "min":"station_monthly_min"})
threshold = threshold[['key', 'iqr_upper_bound', 'iqr_lower_bound']]

sr = sr.merge(threshold, on='key', how='left')
sr = sr[(sr.LS_Temp < sr.iqr_upper_bound) & (sr.LS_Temp > sr.iqr_lower_bound)]

sr['rn_sr'] = sr.index
sr.to_csv("sr.csv", index = False)
sr[['rn_sr', 'StationCode', 'datetime']].to_csv("sr_rn.csv", index = False)
sr

In [2]:
sr = pd.read_csv("sr.csv")
obs= pd.read_csv("obs_filtered.csv")
station_list = obs.StationCode.unique()
for i in station_list:
    station_sr = sr[sr.StationCode==i]
    station_obs = obs[obs.StationCode==i]
    station_matchup =  pd.merge(station_sr, station_obs, on='StationCode', how='inner')
    station_matchup['datetime'] = pd.to_datetime(station_matchup['datetime'])
    station_matchup['unitysampledatetime'] = pd.to_datetime(station_matchup['unitysampledatetime'])

    station_matchup['diff_minute'] = (station_matchup['datetime']- station_matchup['unitysampledatetime']).abs()
    station_matchup['diff_minute'] = (station_matchup['diff_minute'].dt.total_seconds())/60
    station_matchup = station_matchup[station_matchup.diff_minute < 60]
    station_matchup.to_csv(str(i)+'.csv', index = False)

In [11]:
path = r"D:\Estuary Temperature\insitu data\final\matchup"
matchup = mergeCSV(path)
matchup = matchup.drop_duplicates(subset=['date', 'LS_Temp', 'Temp', 'siteID'])
matchup['diff_temp'] = np.abs(matchup['LS_Temp'] - matchup['Temp'])

matchup['year'] = matchup['year_x'] 
matchup['month'] = matchup['month_x'] 

siteInfo = pd.read_csv(r"D:\Estuary Temperature\insitu data\old 1\timezone.csv")
siteInfo['Name'] = siteInfo['Name'].str.lower()
siteInfo = siteInfo[['Name', 'long','lat']].drop_duplicates()
siteInfo = siteInfo.rename(columns={"Name": "StationCode"})

matchup = matchup.merge(siteInfo, how='left', on='StationCode')


matchup = matchup[['aerosol', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'TIR1','sd_NirSD', 'pixel_qa', 'clouds', 'ST_CDIST', 'ST_QA',  'time', 
                  'sat', 'pathrow','date', 'datetime', 'StationCode', 'year', 'month','LS_Temp',  'sampleddatetime','station_monthly_max', 'station_monthly_min', 
                   'Temp','unitysampledatetime', 'diff_minute','long','lat']]

matchup.to_csv('matchup.csv', index=False) 
matchup['key'] = matchup['StationCode'].astype(str) + matchup['date'].astype(str)
value = matchup.groupby(['key'])['diff_minute'].min().reset_index()
matchup = matchup.merge(value, on='key', how='left')
matchup['threshold'] = matchup['diff_minute_x'] - matchup['diff_minute_y']
matchup = matchup[matchup['threshold'] ==0]
#matchup = matchup[['LS_Temp', 'Temp', 'sat', 'year', 'month','lat','long','StationCode']]
#matchup.to_csv('matchup.csv', index=False)

In [12]:
matchup 

,aerosol,blue,green,red,nir,swir1,swir2,TIR1,sd_NirSD,pixel_qa,...,station_monthly_max,station_monthly_min,Temp,unitysampledatetime,diff_minute_x,long,lat,key,diff_minute_y,threshold
3,-99.000000,0.041037,0.052835,0.055420,0.048462,0.033200,0.028745,298.107704,0.004923,5504.0,...,23.8,16.4,21.7,2015-04-26 15:45:00,8.499883,-80.3247,32.5040,aceeiwq2015-04-26,8.499883,0.0
7,-99.000000,0.041037,0.052835,0.055420,0.048462,0.033200,0.028745,298.107704,0.004923,5504.0,...,23.8,16.4,21.7,2015-04-26 15:45:00,8.499883,-80.3247,32.5040,aceeiwq2015-04-26,8.499883,0.0
10,-99.000000,0.056355,0.071563,0.066722,0.065458,0.047665,0.036995,300.777178,0.007689,5504.0,...,28.1,20.0,27.8,2015-05-28 15:30:00,23.702717,-80.3247,32.5040,aceeiwq2015-05-28,23.702717,0.0
13,-99.000000,0.056355,0.071563,0.066722,0.065458,0.047665,0.036995,300.777178,0.007689,5504.0,...,28.1,20.0,27.8,2015-05-28 15:30:00,23.702717,-80.3247,32.5040,aceeiwq2015-05-28,23.702717,0.0
15,-99.000000,0.061580,0.080005,0.072112,0.068977,0.049892,0.037847,302.752794,0.008366,5504.0,...,31.2,26.5,28.4,2015-06-13 15:45:00,8.830700,-80.3247,32.5040,aceeiwq2015-06-13,8.830700,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107862,0.012987,0.020880,0.036170,0.029542,0.022475,0.023767,0.019725,301.587249,0.003973,21952.0,...,28.2,21.8,27.0,2022-07-30 15:30:00,2.477133,-70.5084,41.5542,wqbslwq2022-07-30,2.477133,0.0
107866,0.009412,0.013647,0.020137,0.011090,0.002634,0.002936,0.002524,287.739141,0.002178,21952.0,...,17.3,2.5,13.7,2022-11-03 15:30:00,2.241917,-70.5084,41.5542,wqbslwq2022-11-03,2.241917,0.0
107870,0.009412,0.013647,0.020137,0.011090,0.002634,0.002936,0.002524,287.739141,0.002178,21952.0,...,17.3,2.5,13.7,2022-11-03 15:30:00,2.241917,-70.5084,41.5542,wqbslwq2022-11-03,2.241917,0.0
107874,0.011654,0.015146,0.023272,0.013304,0.003899,0.003445,0.002386,277.732887,0.003302,21952.0,...,17.3,2.5,6.2,2022-11-19 15:15:00,12.636683,-70.5084,41.5542,wqbslwq2022-11-19,12.636683,0.0


In [10]:
matchup = matchup[['LS_Temp', 'Temp', 'sat', 'year', 'month','date','lat','long','StationCode']]
matchup['diff_temp'] = np.abs(matchup['LS_Temp'] -matchup['Temp'] )
matchup = matchup[matchup['diff_temp'] < (matchup['diff_temp'].quantile(.98))]
matchup

,LS_Temp,Temp,sat,year,month,date,lat,long,StationCode,diff_temp
3,24.957704,21.7,LE07,2015,4,2015-04-26,32.5040,-80.3247,aceeiwq,3.257704
7,24.957704,21.7,LE07,2015,4,2015-04-26,32.5040,-80.3247,aceeiwq,3.257704
10,27.627178,27.8,LE07,2015,5,2015-05-28,32.5040,-80.3247,aceeiwq,0.172822
13,27.627178,27.8,LE07,2015,5,2015-05-28,32.5040,-80.3247,aceeiwq,0.172822
15,29.602794,28.4,LE07,2015,6,2015-06-13,32.5040,-80.3247,aceeiwq,1.202794
...,...,...,...,...,...,...,...,...,...,...
107862,28.437249,27.0,LC08,2022,7,2022-07-30,41.5542,-70.5084,wqbslwq,1.437249
107866,14.589141,13.7,LC08,2022,11,2022-11-03,41.5542,-70.5084,wqbslwq,0.889141
107870,14.589141,13.7,LC08,2022,11,2022-11-03,41.5542,-70.5084,wqbslwq,0.889141
107874,4.582887,6.2,LC08,2022,11,2022-11-19,41.5542,-70.5084,wqbslwq,1.617113


In [56]:
matchup['year_month'] = matchup['year']+matchup['month']
matchup['lat_long'] = matchup.lat.round(decimals = 0).astype(str) + "_" + matchup.long.round(decimals = 0).astype(str)

train = matchup.groupby(['lat_long','year_month']).sample(frac=.80, random_state=2023)
mask = ~matchup.index.isin(train.index)
gems = pd.read_csv("D:\Estuary Temperature\insitu data\gems\gems.csv")
validation = pd.concat([matchup[mask], gems])
print(train.shape)
print(validation.shape)

validation.to_csv('validation.csv', index=False)
train.to_csv('train.csv', index=False)

(23232, 12)
(5824, 12)
